# **Rozwiązanie na konkurs pt. Titanic.  Damian Strzyż 215349**
# Notebook jest wynikiem pracy nad rozwiązaniem do konkursu "Titanic" na stronie kaggle.com. Model polega na przewidywaniu, który z pasażerów statku przeżył katastrofe. Model opiera się na bazie danych uczących "train_data" i bazie danych do testowania wyników modelu "test_data".

In [1]:
import numpy as np
import pandas as pd

W powyższej komórce programu importowane zostają potrzebne do dalszej pracy biblioteki

In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

Następnie zostają zaimportowane bazy danych do uczenia i do testowania

In [3]:
train_data[['Cabin', 'Survived']].groupby(['Cabin']).mean()

,Survived
Cabin,
A10,0.0
A14,0.0
A16,1.0
A19,0.0
A20,1.0
...,...
F33,1.0
F38,0.0
F4,1.0


Dane zawierające numer kabiny są nie do końca jednoznacze przez co wykluczyłem je jako dane do modelu

In [4]:
train_data[['Fare', 'Survived']].groupby(['Fare']).mean()

,Survived
Fare,
0.0000,0.066667
4.0125,0.000000
5.0000,0.000000
6.2375,0.000000
6.4375,0.000000
...,...
227.5250,0.750000
247.5208,0.500000
262.3750,1.000000


Podobnie jak w przypadku danych o numerach kabin, opłaty za bilet nie są jednoznaczne, ponieważ wyższa opłata nie oznaczała większej przeżywalności

In [5]:
train_data[['Age', 'Survived']].groupby(['Age']).mean()

,Survived
Age,
0.42,1.0
0.67,1.0
0.75,1.0
0.83,1.0
0.92,1.0
...,...
70.00,0.0
70.50,0.0
71.00,0.0


Wiek jest postanowiłem podzielić na mniejsze grupy z racji tego, że lepiej one opisują trend przeżywalności

In [6]:
train_data[['Pclass', 'Survived']].groupby(['Pclass']).mean()

,Survived
Pclass,
1,0.629630
2,0.472826
3,0.242363


Dane opisujące klasę w której podróżował pasażer jednoznacznie pokazują trend wzrostu przeżywalności wraz ze wzrostem klasy

In [7]:
train_data[['SibSp', 'Survived']].groupby(['SibSp']).mean()

,Survived
SibSp,
0,0.345395
1,0.535885
2,0.464286
3,0.250000
4,0.166667
5,0.000000
8,0.000000


In [8]:
train_data[['Parch', 'Survived']].groupby(['Parch']).mean()

,Survived
Parch,
0,0.343658
1,0.550847
2,0.500000
3,0.600000
4,0.000000
5,0.200000
6,0.000000


Powyższe dwie grupy danych opisujące ilości krewnych na statku pokazują słabszą przeżywalność większych rodzin

In [9]:
train_data['Age'] = train_data['Age'].fillna(42)
train_data['Age_groups'] = pd.cut(train_data['Age'],10,labels=[0,1,2,3,4,5,6,7,8,9], ordered=False)
train_data[['Age_groups', 'Survived']].groupby(['Age_groups']).mean()

,Survived
Age_groups,
0,0.680000
1,0.380952
2,0.370370
3,0.342857
4,0.416667
5,0.445545
6,0.301310
7,0.458333
8,0.392857


In [10]:
test_data['Age'] = test_data['Age'].fillna(42)
test_data['Age_groups'] = pd.cut(test_data['Age'],12,labels=[0,1,2,3,4,5,6,7,8,9,10,11], ordered=False)

Uzupełniam puste informacje o wieku średnią wieku, a także dziele ludzi na grupy wiekowe tak jak pisałem wyżej

In [11]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch", "Age_groups"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


# W ostatniej komórce został stworzony model na podstawie grup danych wybranych przezemnie na drodze analizy poszczególnych wyników. Do stworzenia modelu użylem RandomForestClassifier, poczym wynik modelu na zestawie danych testowych został zapisany w pliku "my_submission.csv". Wynik to 0.78229 co daje 3581 miejsce.